In [94]:
import pandas as pd
import numpy as np

In [95]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [96]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [97]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


In [98]:
Flugbewegungen.head()



,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [99]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [100]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [101]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [102]:

Flugbewegungen["Datum"].dtype
Wetterdaten["Datum"].dtype

dtype('O')

In [103]:
                                                                                                # The merge isnt correct because it erases the Hour format and therefore setting the entire Day to one value?!
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
                      .merge(Flugbewegungen, on='Datum', how='inner') \

#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(20)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic,Weekday
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,0,-1.142151,Saturday
1,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,1,-1.295479,Saturday
2,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,2,-1.295479,Saturday
3,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,3,-1.295479,Saturday
4,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,4,-1.295479,Saturday
5,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,5,-1.295479,Saturday
6,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,6,-0.682167,Saturday
7,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,7,-0.222182,Saturday
8,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,8,-0.835495,Saturday
9,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,9,-0.528839,Saturday


In [104]:
print(final_df.tail(25))

                           Datum  Rebgassechange  Clarahuuschange  Citychange  \
24575  2024-10-20 00:00:00+00:00       -0.139856        -0.000213   -0.076628   
24576  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24577  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24578  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24579  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24580  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24581  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24582  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24583  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24584  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24585  2024-10-21 00:00:00+00:00        0.000125        -0.000213   -0.011259   
24586  2024-10-21 00:00:00+0

In [105]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 14:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-22 00:00:00+00:00
2024-11-24 04:00:00+00:00
2024-11-20 22:00:00+00:00
2024-11-19 22:00:00+00:00


# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [106]:
type(final_df[["Datum"]])

pandas.core.frame.DataFrame

In [107]:
final_df.drop(columns=["Weekday"], inplace=True)

In [108]:
final_df.to_csv("../Working_DataFrame.csv", index=False)


#final_df.tail(40)
final_df['Datum'].dtype
import numpy as np

cutoff = np.datetime64('2024-11-20 10:00:00')
final_df = final_df[final_df['Datum'] <= cutoff]
final_df.tail(40)

In [109]:
Flugbewegungen.head()

,Datum,Hour,Traffic,Weekday
0,2022-01-01 00:00:00+00:00,0,-1.142151,Saturday
1,2022-01-01 00:00:00+00:00,1,-1.295479,Saturday
2,2022-01-01 00:00:00+00:00,2,-1.295479,Saturday
3,2022-01-01 00:00:00+00:00,3,-1.295479,Saturday
4,2022-01-01 00:00:00+00:00,4,-1.295479,Saturday


In [110]:

Stromverbrauch["Datum"].dtype
Wetterdaten["Datum"].dtype
Gasverbrauch["Datum"].dtype

Flugbewegungen["Datum"] = Flugbewegungen["Datum"].astype(str)
Flugbewegungen["Datum"].dtype

dtype('O')

In [111]:
#DF NN
DF_NN = Gasverbrauch.merge(Stromverbrauch, on='Datum', how='left') \
                      .merge(Wetterdaten, on='Datum', how='left') \
                      .merge(Flugbewegungen, on='Datum', how='left')

In [112]:
DF_NN.head()

# Convert the date column to datetime format
DF_NN['Datum'] = pd.to_datetime(DF_NN['Datum'])

# Filter the dataframe to keep only dates before the specified date and time
DF_NN = DF_NN[DF_NN['Datum'] <= '2024-10-27 13:00']
DF_NN.head()


,Gasverbrauch,Datum,Stromverbrauch,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m],Hour,Traffic,Weekday
0,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,0.0,-1.142151,Saturday
1,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,1.0,-1.295479,Saturday
2,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,2.0,-1.295479,Saturday
3,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,3.0,-1.295479,Saturday
4,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,4.0,-1.295479,Saturday


In [113]:

DF_NN.drop(columns=["Datum", "Hour"], inplace=True)

DF_NN.to_csv("./Working_DataFrame_NN.csv", index=False)

In [114]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen


In [115]:
final_df.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,Hour,Traffic
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,0,-1.142151
1,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,1,-1.295479
2,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,2,-1.295479
3,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,3,-1.295479
4,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.15127,...,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,4,-1.295479


In [116]:
import pandas as pd
df = final_df.copy()
# Ensure 'Datum' is a datetime object
df['Datum'] = pd.to_datetime(df['Datum'])

# Create a complete range of hourly timestamps from the first to the last point
full_range = pd.date_range(start=df['Datum'].min(), end=df['Datum'].max(), freq='h')

# Identify missing timestamps
missing_timestamps = full_range.difference(df['Datum'])

# Display the results
if missing_timestamps.empty:
    print("No missing hourly timestamps. All hours are accounted for.")
else:
    print("Missing hourly timestamps:")
    print(missing_timestamps)

Missing hourly timestamps:
DatetimeIndex(['2022-01-01 01:00:00+00:00', '2022-01-01 02:00:00+00:00',
               '2022-01-01 03:00:00+00:00', '2022-01-01 04:00:00+00:00',
               '2022-01-01 05:00:00+00:00', '2022-01-01 06:00:00+00:00',
               '2022-01-01 07:00:00+00:00', '2022-01-01 08:00:00+00:00',
               '2022-01-01 09:00:00+00:00', '2022-01-01 10:00:00+00:00',
               ...
               '2024-10-20 14:00:00+00:00', '2024-10-20 15:00:00+00:00',
               '2024-10-20 16:00:00+00:00', '2024-10-20 17:00:00+00:00',
               '2024-10-20 18:00:00+00:00', '2024-10-20 19:00:00+00:00',
               '2024-10-20 20:00:00+00:00', '2024-10-20 21:00:00+00:00',
               '2024-10-20 22:00:00+00:00', '2024-10-20 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=23552, freq=None)
